## 1. Import Libs

In [1]:
import torch
from torch.autograd import Variable
from torchvision import transforms
from dataset import DatasetFromFolder
from model import Generator, Discriminator

import utils
import argparse
import os, itertools
from logger import Logger
import numpy as np

/home/ncp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2. Setting Hyperparameters

In [2]:
parser = argparse.ArgumentParser()

#Data Set Parameter
parser.add_argument('--dataset', required=False, default='horse2zebra', help='input dataset')
parser.add_argument('--batch_size', type=int, default=1, help='train batch size')
parser.add_argument('--input_size', type=int, default=256, help='input size')
parser.add_argument('--resize_scale', type=int, default=286, help='resize scale (0 is false)')
parser.add_argument('--crop_size', type=int, default=256, help='crop size (0 is false)')
parser.add_argument('--fliplr', type=bool, default=True, help='random fliplr True of False')

#Model Parameters 
parser.add_argument('--ngf', type=int, default=32) # number of generator filters
parser.add_argument('--ndf', type=int, default=64) # number of discriminator filters
parser.add_argument('--num_resnet', type=int, default=6, help='number of resnet blocks in generator')

#Learning Parameters
parser.add_argument('--num_epochs', type=int, default=200, help='number of train epochs')
parser.add_argument('--decay_epoch', type=int, default=100, help='start decaying learning rate after this number')
parser.add_argument('--lrG', type=float, default=0.0002, help='learning rate for generator, default=0.0002')
parser.add_argument('--lrD', type=float, default=0.0002, help='learning rate for discriminator, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
parser.add_argument('--lambdaA', type=float, default=10, help='lambdaA for cycle loss')
parser.add_argument('--lambdaB', type=float, default=10, help='lambdaB for cycle loss')
params = parser.parse_args([])
print(params)

# Directories for loading data and saving results
data_dir = 'data/' + params.dataset + '/'
save_dir = params.dataset + '_results/'
model_dir = params.dataset + '_model/'

# Set the logger
D_A_log_dir = save_dir + 'D_A_logs'
D_B_log_dir = save_dir + 'D_B_logs'
D_A_logger = Logger(D_A_log_dir)
D_B_logger = Logger(D_B_log_dir)

G_A_log_dir = save_dir + 'G_A_logs'
G_B_log_dir = save_dir + 'G_B_logs'

G_A_logger = Logger(G_A_log_dir)
G_B_logger = Logger(G_B_log_dir)

cycle_A_log_dir = save_dir + 'cycle_A_logs'
cycle_B_log_dir = save_dir + 'cycle_B_logs'

cycle_A_logger = Logger(cycle_A_log_dir)
cycle_B_logger = Logger(cycle_B_log_dir)

img_log_dir = save_dir + 'img_logs'
img_logger = Logger(img_log_dir)

Namespace(batch_size=1, beta1=0.5, beta2=0.999, crop_size=256, dataset='horse2zebra', decay_epoch=100, fliplr=True, input_size=256, lambdaA=10, lambdaB=10, lrD=0.0002, lrG=0.0002, ndf=64, ngf=32, num_epochs=200, num_resnet=6, resize_scale=286)


## 3. Load Dataset
### 3.2 Preprocessing

In [3]:
transform = transforms.Compose([
    transforms.Scale((params.input_size,params.input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

### 3.2 Load Train Data

In [4]:
train_data_A = DatasetFromFolder(data_dir, subfolder='trainA', transform=transform,
                                resize_scale=params.resize_scale, crop_size=params.crop_size, fliplr=params.fliplr)

train_data_loader_A = torch.utils.data.DataLoader(dataset=train_data_A, batch_size=params.batch_size, shuffle=True)

train_data_B = DatasetFromFolder(data_dir, subfolder='trainB', transform=transform,
                                resize_scale=params.resize_scale, crop_size=params.crop_size, fliplr=params.fliplr)

train_data_loader_B = torch.utils.data.DataLoader(dataset=train_data_B, batch_size=params.batch_size, shuffle=True)

### 3.3 Load Test Data

In [5]:
test_data_A = DatasetFromFolder(data_dir, subfolder='testA', transform=transform)

test_data_loader_A = torch.utils.data.DataLoader(dataset=test_data_A, batch_size=params.batch_size, shuffle=False)

test_data_B = DatasetFromFolder(data_dir, subfolder='testB', transform=transform)

test_data_loader_B = torch.utils.data.DataLoader(dataset=test_data_B, batch_size=params.batch_size, shuffle=False)


# Get specific test images
test_real_A_data = train_data_A.__getitem__(11).unsqueeze(0) # Convert to 4d tensor (BxNxHxW)
test_real_B_data = train_data_B.__getitem__(91).unsqueeze(0)
print(test_real_A_data)


( 0 , 0 ,.,.) = 
  0.3490  0.2784  0.3333  ...   0.5686  0.5529  0.4902
  0.3412  0.3412  0.3569  ...   0.4510  0.4510  0.3725
  0.3176  0.3176  0.2314  ...   0.3961  0.3804  0.2941
           ...             ⋱             ...          
 -0.3098 -0.5451 -0.5216  ...  -0.1922 -0.2392 -0.1137
 -0.2314 -0.4196 -0.4980  ...  -0.0353 -0.0353 -0.0275
 -0.3176 -0.3412 -0.3882  ...  -0.0588  0.0902  0.0902

( 0 , 1 ,.,.) = 
  0.2627  0.1843  0.2471  ...   0.4980  0.4667  0.4196
  0.2471  0.2549  0.2784  ...   0.3961  0.3804  0.3020
  0.2157  0.2314  0.1686  ...   0.3490  0.3333  0.2471
           ...             ⋱             ...          
 -0.1843 -0.4353 -0.4196  ...  -0.1608 -0.1137  0.0118
 -0.1216 -0.3569 -0.4745  ...  -0.0118  0.0510  0.0196
 -0.2078 -0.2863 -0.4039  ...  -0.0275  0.1137  0.0039

( 0 , 2 ,.,.) = 
  0.0353  0.0196  0.1059  ...   0.0667  0.0510 -0.0118
  0.0431  0.0902  0.1373  ...  -0.0667 -0.0667 -0.1294
  0.0431  0.0902  0.0431  ...  -0.1373 -0.1451 -0.2078
           

## 4. Build Model & Optimizers & Criterions
### 4.1 Build Model

In [6]:
G_A = Generator(3, params.ngf, 3, params.num_resnet) # input_dim, num_filter, output_dim, num_resnet
G_B = Generator(3, params.ngf, 3, params.num_resnet)

D_A = Discriminator(3, params.ndf, 1) # input_dim, num_filter, output_dim
D_B = Discriminator(3, params.ndf, 1)

G_A.normal_weight_init(mean=0.0, std=0.02)
G_B.normal_weight_init(mean=0.0, std=0.02)
D_A.normal_weight_init(mean=0.0, std=0.02)
D_B.normal_weight_init(mean=0.0, std=0.02)

print(G_A.cuda())
print(G_B.cuda())
print(D_A.cuda())
print(D_B.cuda())

Generator(
  (pad): ReflectionPad2d((3, 3, 3, 3))
  (conv1): ConvBlock(
    (conv): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1))
    (bn): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False)
    (relu): ReLU(inplace)
    (lrelu): LeakyReLU(0.2, inplace)
    (tanh): Tanh()
  )
  (conv2): ConvBlock(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False)
    (relu): ReLU(inplace)
    (lrelu): LeakyReLU(0.2, inplace)
    (tanh): Tanh()
  )
  (conv3): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False)
    (relu): ReLU(inplace)
    (lrelu): LeakyReLU(0.2, inplace)
    (tanh): Tanh()
  )
  (resnet_blocks): Sequential(
    (0): ResnetBlock(
      (resnet_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
        

### 4.2 Optimizers

In [7]:
G_optimizer = torch.optim.Adam(itertools.chain(G_A.parameters(), G_B.parameters()), lr=params.lrG, betas=(params.beta1, params.beta2))
D_A_optimizer = torch.optim.Adam(D_A.parameters(), lr=params.lrD, betas=(params.beta1, params.beta2))
D_B_optimizer = torch.optim.Adam(D_B.parameters(), lr=params.lrD, betas=(params.beta1, params.beta2))

### 4.3 Loss Functions

In [8]:
MSE_Loss = torch.nn.MSELoss().cuda()
L1_Loss = torch.nn.L1Loss().cuda()

# # Training GAN
D_A_avg_losses = []
D_B_avg_losses = []
G_A_avg_losses = []
G_B_avg_losses = []
cycle_A_avg_losses = []
cycle_B_avg_losses = []

# Generated image pool
num_pool = 50
fake_A_pool = utils.ImagePool(num_pool)
fake_B_pool = utils.ImagePool(num_pool)

## 5. Training Models

In [9]:
step = 0
for epoch in range(params.num_epochs):
    D_A_losses = []
    D_B_losses = []
    G_A_losses = []
    G_B_losses = []
    cycle_A_losses = []
    cycle_B_losses = []
    
    # Learing rate decay 설정 구간
    if(epoch + 1) > params.decay_epoch:
        D_A_optimizer.param_groups[0]['lr'] -= params.lrD / (params.num_epochs - params.decay_epoch)
        D_B_optimizer.param_groups[0]['lr'] -= params.lrD / (params.num_epochs - params.decay_epoch)
        G_optimizer.param_groups[0]['lr'] -= params.lrG / (params.num_epochs - params.decay_epoch)
        
    
    # training 구간
    for i, (real_A, real_B) in enumerate(zip(train_data_loader_A, train_data_loader_B)):
        
        # input image data
        real_A = Variable(real_A.cuda())
        real_B = Variable(real_B.cuda())
        
        # -------------------------- train generator G --------------------------
        # A --> B
        fake_B = G_A(real_A)
        D_B_fake_decision = D_B(fake_B)
        G_A_loss = MSE_Loss(D_B_fake_decision, Variable(torch.ones(D_B_fake_decision.size()).cuda()))
        
        # forward cycle loss
        recon_A = G_B(fake_B)
        cycle_A_loss = L1_Loss(recon_A, real_A) * params.lambdaA
        
        # B --> A
        fake_A = G_B(real_B)
        D_A_fake_decision = D_A(fake_A)
        G_B_loss = MSE_Loss(D_A_fake_decision, Variable(torch.ones(D_A_fake_decision.size()).cuda()))
        
        # backward cycle loss
        recon_B = G_A(fake_A)
        cycle_B_loss = L1_Loss(recon_B, real_B) * params.lambdaB
        
        # Back propagation
        G_loss = G_A_loss + G_B_loss + cycle_A_loss + cycle_B_loss
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        
        
        # -------------------------- train discriminator D_A --------------------------
        D_A_real_decision = D_A(real_A)
        D_A_real_loss = MSE_Loss(D_A_real_decision, Variable(torch.ones(D_A_real_decision.size()).cuda()))
        
        fake_A = fake_A_pool.query(fake_A)
        
        D_A_fake_decision = D_A(fake_A)
        D_A_fake_loss = MSE_Loss(D_A_fake_decision, Variable(torch.zeros(D_A_fake_decision.size()).cuda()))
        
        # Back propagation
        D_A_loss = (D_A_real_loss + D_A_fake_loss) * 0.5
        D_A_optimizer.zero_grad()
        D_A_loss.backward()
        D_A_optimizer.step()
        
        # -------------------------- train discriminator D_B --------------------------
        D_B_real_decision = D_B(real_B)
        D_B_real_loss = MSE_Loss(D_B_real_decision, Variable(torch.ones(D_B_fake_decision.size()).cuda()))
        
        fake_B = fake_B_pool.query(fake_B)
        
        D_B_fake_decision = D_B(fake_B)
        D_B_fake_loss = MSE_Loss(D_B_fake_decision, Variable(torch.zeros(D_B_fake_decision.size()).cuda()))
        
        # Back propagation
        D_B_loss = (D_B_real_loss + D_B_fake_loss) * 0.5
        D_B_optimizer.zero_grad()
        D_B_loss.backward()
        D_B_optimizer.step()
        
        # ------------------------ Print -----------------------------
        # loss values
        D_A_losses.append(D_A_loss.data[0])
        D_B_losses.append(D_B_loss.data[0])
        G_A_losses.append(G_A_loss.data[0])
        G_B_losses.append(G_B_loss.data[0])
        cycle_A_losses.append(cycle_A_loss.data[0])
        cycle_B_losses.append(cycle_B_loss.data[0])

        if i%10 == 0:
            print('Epoch [%d/%d], Step [%d/%d], D_A_loss: %.4f, D_B_loss: %.4f, G_A_loss: %.4f, G_B_loss: %.4f'
                  % (epoch+1, params.num_epochs, i+1, len(train_data_loader_A), D_A_loss.data[0], D_B_loss.data[0], G_A_loss.data[0], G_B_loss.data[0]))
            
        # ============ TensorBoard logging ============#
        D_A_logger.scalar_summary('losses', D_A_loss.data[0], step + 1)
        D_B_logger.scalar_summary('losses', D_B_loss.data[0], step + 1)
        G_A_logger.scalar_summary('losses', G_A_loss.data[0], step + 1)
        G_B_logger.scalar_summary('losses', G_B_loss.data[0], step + 1)
        cycle_A_logger.scalar_summary('losses', cycle_A_loss.data[0], step + 1)
        cycle_B_logger.scalar_summary('losses', cycle_B_loss.data[0], step + 1)
        step += 1
        
    D_A_avg_loss = torch.mean(torch.FloatTensor(D_A_losses))
    D_B_avg_loss = torch.mean(torch.FloatTensor(D_B_losses))
    G_A_avg_loss = torch.mean(torch.FloatTensor(G_A_losses))
    G_B_avg_loss = torch.mean(torch.FloatTensor(G_B_losses))
    cycle_A_avg_loss = torch.mean(torch.FloatTensor(cycle_A_losses))
    cycle_B_avg_loss = torch.mean(torch.FloatTensor(cycle_B_losses))

    # avg loss values for plot
    D_A_avg_losses.append(D_A_avg_loss)
    D_B_avg_losses.append(D_B_avg_loss)
    G_A_avg_losses.append(G_A_avg_loss)
    G_B_avg_losses.append(G_B_avg_loss)
    cycle_A_avg_losses.append(cycle_A_avg_loss)
    cycle_B_avg_losses.append(cycle_B_avg_loss)

    # Show result for test image
    test_real_A = Variable(test_real_A_data.cuda())
    test_fake_B = G_A(test_real_A)
    test_recon_A = G_B(test_fake_B)

    test_real_B = Variable(test_real_B_data.cuda())
    test_fake_A = G_B(test_real_B)
    test_recon_B = G_A(test_fake_A)

    utils.plot_train_result([test_real_A, test_real_B], [test_fake_B, test_fake_A], [test_recon_A, test_recon_B],
                            epoch, save=True, save_dir=save_dir)

    # log the images
    result_AtoB = np.concatenate((utils.to_np(test_real_A), utils.to_np(test_fake_B), utils.to_np(test_recon_A)), axis=3)
    result_BtoA = np.concatenate((utils.to_np(test_real_B), utils.to_np(test_fake_A), utils.to_np(test_recon_B)), axis=3)

    info = { 'result_AtoB': result_AtoB.transpose(0, 2, 3, 1),  # convert to BxHxWxC
             'result_BtoA': result_BtoA.transpose(0, 2, 3, 1) }

    for tag, images in info.items():
        img_logger.image_summary(tag, images, epoch + 1)

Epoch [1/200], Step [1/1067], D_A_loss: 0.5368, D_B_loss: 0.5034, G_A_loss: 0.8002, G_B_loss: 0.9582
Epoch [1/200], Step [11/1067], D_A_loss: 0.3148, D_B_loss: 0.3428, G_A_loss: 0.3399, G_B_loss: 0.2891
Epoch [1/200], Step [21/1067], D_A_loss: 0.2535, D_B_loss: 0.2348, G_A_loss: 0.3818, G_B_loss: 0.2847
Epoch [1/200], Step [31/1067], D_A_loss: 0.3186, D_B_loss: 0.3931, G_A_loss: 0.7738, G_B_loss: 0.3859
Epoch [1/200], Step [41/1067], D_A_loss: 0.3415, D_B_loss: 0.2320, G_A_loss: 0.3605, G_B_loss: 0.4218
Epoch [1/200], Step [51/1067], D_A_loss: 0.2629, D_B_loss: 0.2178, G_A_loss: 0.5693, G_B_loss: 0.4091
Epoch [1/200], Step [61/1067], D_A_loss: 0.2171, D_B_loss: 0.3190, G_A_loss: 0.4678, G_B_loss: 0.2745
Epoch [1/200], Step [71/1067], D_A_loss: 0.2923, D_B_loss: 0.2944, G_A_loss: 0.2868, G_B_loss: 0.4338
Epoch [1/200], Step [81/1067], D_A_loss: 0.2180, D_B_loss: 0.2870, G_A_loss: 0.3484, G_B_loss: 0.3156
Epoch [1/200], Step [91/1067], D_A_loss: 0.2218, D_B_loss: 0.1730, G_A_loss: 0.2867

Epoch [1/200], Step [801/1067], D_A_loss: 0.2576, D_B_loss: 0.2407, G_A_loss: 0.2695, G_B_loss: 0.6254
Epoch [1/200], Step [811/1067], D_A_loss: 0.3679, D_B_loss: 0.0984, G_A_loss: 0.3395, G_B_loss: 0.0524
Epoch [1/200], Step [821/1067], D_A_loss: 0.3162, D_B_loss: 0.2332, G_A_loss: 0.2801, G_B_loss: 0.4361
Epoch [1/200], Step [831/1067], D_A_loss: 0.0776, D_B_loss: 0.0477, G_A_loss: 0.7011, G_B_loss: 0.3523
Epoch [1/200], Step [841/1067], D_A_loss: 0.1131, D_B_loss: 0.2342, G_A_loss: 0.8285, G_B_loss: 0.5045
Epoch [1/200], Step [851/1067], D_A_loss: 0.1211, D_B_loss: 0.1155, G_A_loss: 0.3278, G_B_loss: 0.1160
Epoch [1/200], Step [861/1067], D_A_loss: 0.1899, D_B_loss: 0.4503, G_A_loss: 0.0608, G_B_loss: 0.1965
Epoch [1/200], Step [871/1067], D_A_loss: 0.0961, D_B_loss: 0.2628, G_A_loss: 0.1353, G_B_loss: 0.5472
Epoch [1/200], Step [881/1067], D_A_loss: 0.1340, D_B_loss: 0.2116, G_A_loss: 0.3309, G_B_loss: 0.0975
Epoch [1/200], Step [891/1067], D_A_loss: 0.3133, D_B_loss: 0.1637, G_A_l

Epoch [2/200], Step [531/1067], D_A_loss: 0.1811, D_B_loss: 0.0679, G_A_loss: 0.2855, G_B_loss: 0.2382
Epoch [2/200], Step [541/1067], D_A_loss: 0.2169, D_B_loss: 0.1103, G_A_loss: 0.3301, G_B_loss: 0.4837
Epoch [2/200], Step [551/1067], D_A_loss: 0.2472, D_B_loss: 0.3207, G_A_loss: 0.2495, G_B_loss: 0.2731
Epoch [2/200], Step [561/1067], D_A_loss: 0.2281, D_B_loss: 0.1734, G_A_loss: 0.2960, G_B_loss: 0.1675
Epoch [2/200], Step [571/1067], D_A_loss: 0.3891, D_B_loss: 0.1097, G_A_loss: 0.5749, G_B_loss: 0.2196
Epoch [2/200], Step [581/1067], D_A_loss: 0.2510, D_B_loss: 0.3126, G_A_loss: 0.1026, G_B_loss: 0.4337
Epoch [2/200], Step [591/1067], D_A_loss: 0.1212, D_B_loss: 0.1727, G_A_loss: 0.9362, G_B_loss: 0.0894
Epoch [2/200], Step [601/1067], D_A_loss: 0.1481, D_B_loss: 0.3422, G_A_loss: 0.7425, G_B_loss: 0.2566
Epoch [2/200], Step [611/1067], D_A_loss: 0.1064, D_B_loss: 0.1659, G_A_loss: 0.3530, G_B_loss: 0.5818
Epoch [2/200], Step [621/1067], D_A_loss: 0.2669, D_B_loss: 0.1555, G_A_l

Epoch [3/200], Step [261/1067], D_A_loss: 0.1258, D_B_loss: 0.0542, G_A_loss: 0.5719, G_B_loss: 0.4203
Epoch [3/200], Step [271/1067], D_A_loss: 0.2538, D_B_loss: 0.0348, G_A_loss: 0.4580, G_B_loss: 0.2119
Epoch [3/200], Step [281/1067], D_A_loss: 0.1325, D_B_loss: 0.0548, G_A_loss: 0.2847, G_B_loss: 0.5619
Epoch [3/200], Step [291/1067], D_A_loss: 0.3002, D_B_loss: 0.0688, G_A_loss: 0.4878, G_B_loss: 0.7302
Epoch [3/200], Step [301/1067], D_A_loss: 0.1945, D_B_loss: 0.2908, G_A_loss: 0.4179, G_B_loss: 0.3472
Epoch [3/200], Step [311/1067], D_A_loss: 0.2231, D_B_loss: 0.1046, G_A_loss: 0.4470, G_B_loss: 0.1326
Epoch [3/200], Step [321/1067], D_A_loss: 0.2135, D_B_loss: 0.0741, G_A_loss: 0.9215, G_B_loss: 0.3426
Epoch [3/200], Step [331/1067], D_A_loss: 0.2176, D_B_loss: 0.2348, G_A_loss: 0.1915, G_B_loss: 0.3162
Epoch [3/200], Step [341/1067], D_A_loss: 0.0736, D_B_loss: 0.3289, G_A_loss: 0.0835, G_B_loss: 0.1437
Epoch [3/200], Step [351/1067], D_A_loss: 0.1825, D_B_loss: 0.0749, G_A_l

Epoch [3/200], Step [1061/1067], D_A_loss: 0.0760, D_B_loss: 0.2432, G_A_loss: 0.1990, G_B_loss: 0.2257
Epoch [4/200], Step [1/1067], D_A_loss: 0.0624, D_B_loss: 0.1940, G_A_loss: 0.1750, G_B_loss: 0.3348
Epoch [4/200], Step [11/1067], D_A_loss: 0.2529, D_B_loss: 0.1560, G_A_loss: 0.2617, G_B_loss: 0.4699
Epoch [4/200], Step [21/1067], D_A_loss: 0.2491, D_B_loss: 0.2081, G_A_loss: 0.1731, G_B_loss: 0.3381
Epoch [4/200], Step [31/1067], D_A_loss: 0.2041, D_B_loss: 0.1404, G_A_loss: 0.5391, G_B_loss: 0.3988
Epoch [4/200], Step [41/1067], D_A_loss: 0.2115, D_B_loss: 0.3499, G_A_loss: 0.0703, G_B_loss: 0.5230
Epoch [4/200], Step [51/1067], D_A_loss: 0.1759, D_B_loss: 0.1154, G_A_loss: 0.4827, G_B_loss: 0.7560
Epoch [4/200], Step [61/1067], D_A_loss: 0.1046, D_B_loss: 0.0959, G_A_loss: 0.4855, G_B_loss: 0.3076
Epoch [4/200], Step [71/1067], D_A_loss: 0.1765, D_B_loss: 0.2665, G_A_loss: 0.9343, G_B_loss: 0.2352
Epoch [4/200], Step [81/1067], D_A_loss: 0.2820, D_B_loss: 0.0911, G_A_loss: 0.57

...

Epoch [192/200], Step [31/1067], D_A_loss: 0.0759, D_B_loss: 0.1020, G_A_loss: 0.7996, G_B_loss: 0.9954
Epoch [192/200], Step [41/1067], D_A_loss: 0.0302, D_B_loss: 0.0120, G_A_loss: 0.9976, G_B_loss: 0.8978
Epoch [192/200], Step [51/1067], D_A_loss: 0.0377, D_B_loss: 0.0341, G_A_loss: 0.7427, G_B_loss: 0.7310
Epoch [192/200], Step [61/1067], D_A_loss: 0.0402, D_B_loss: 0.0093, G_A_loss: 0.8085, G_B_loss: 0.7717
Epoch [192/200], Step [71/1067], D_A_loss: 0.0529, D_B_loss: 0.0196, G_A_loss: 1.1813, G_B_loss: 0.7428
Epoch [192/200], Step [81/1067], D_A_loss: 0.0955, D_B_loss: 0.0311, G_A_loss: 0.7717, G_B_loss: 0.1211
Epoch [192/200], Step [91/1067], D_A_loss: 0.1103, D_B_loss: 0.0184, G_A_loss: 0.6928, G_B_loss: 0.5804
Epoch [192/200], Step [101/1067], D_A_loss: 0.0682, D_B_loss: 0.0166, G_A_loss: 1.1238, G_B_loss: 0.5339
Epoch [192/200], Step [111/1067], D_A_loss: 0.0852, D_B_loss: 0.0094, G_A_loss: 0.6981, G_B_loss: 0.6082
Epoch [192/200], Step [121/1067], D_A_loss: 0.0740, D_B_loss: 

Epoch [192/200], Step [821/1067], D_A_loss: 0.1298, D_B_loss: 0.0162, G_A_loss: 0.9537, G_B_loss: 0.6252
Epoch [192/200], Step [831/1067], D_A_loss: 0.0232, D_B_loss: 0.0081, G_A_loss: 1.1112, G_B_loss: 1.0620
Epoch [192/200], Step [841/1067], D_A_loss: 0.0805, D_B_loss: 0.0875, G_A_loss: 0.6523, G_B_loss: 0.5513
Epoch [192/200], Step [851/1067], D_A_loss: 0.1204, D_B_loss: 0.0188, G_A_loss: 0.9049, G_B_loss: 0.3473
Epoch [192/200], Step [861/1067], D_A_loss: 0.0189, D_B_loss: 0.0144, G_A_loss: 0.8124, G_B_loss: 0.8680
Epoch [192/200], Step [871/1067], D_A_loss: 0.0692, D_B_loss: 0.0131, G_A_loss: 0.9378, G_B_loss: 0.7830
Epoch [192/200], Step [881/1067], D_A_loss: 0.0353, D_B_loss: 0.0104, G_A_loss: 0.8159, G_B_loss: 0.7669
Epoch [192/200], Step [891/1067], D_A_loss: 0.0231, D_B_loss: 0.0221, G_A_loss: 0.6892, G_B_loss: 0.7821
Epoch [192/200], Step [901/1067], D_A_loss: 0.0380, D_B_loss: 0.0082, G_A_loss: 1.0955, G_B_loss: 0.7134
Epoch [192/200], Step [911/1067], D_A_loss: 0.0321, D_B

Epoch [193/200], Step [541/1067], D_A_loss: 0.0405, D_B_loss: 0.0066, G_A_loss: 1.0300, G_B_loss: 0.8149
Epoch [193/200], Step [551/1067], D_A_loss: 0.0613, D_B_loss: 0.0233, G_A_loss: 1.0689, G_B_loss: 0.9582
Epoch [193/200], Step [561/1067], D_A_loss: 0.0293, D_B_loss: 0.0066, G_A_loss: 0.9271, G_B_loss: 1.1884
Epoch [193/200], Step [571/1067], D_A_loss: 0.0307, D_B_loss: 0.0239, G_A_loss: 0.9832, G_B_loss: 0.7456
Epoch [193/200], Step [581/1067], D_A_loss: 0.0384, D_B_loss: 0.0156, G_A_loss: 1.0202, G_B_loss: 0.9333
Epoch [193/200], Step [591/1067], D_A_loss: 0.1698, D_B_loss: 0.0093, G_A_loss: 0.9374, G_B_loss: 0.4055
Epoch [193/200], Step [601/1067], D_A_loss: 0.0691, D_B_loss: 0.0647, G_A_loss: 0.9553, G_B_loss: 0.5435
Epoch [193/200], Step [611/1067], D_A_loss: 0.0305, D_B_loss: 0.0085, G_A_loss: 0.7673, G_B_loss: 0.8177
Epoch [193/200], Step [621/1067], D_A_loss: 0.0248, D_B_loss: 0.0158, G_A_loss: 0.7396, G_B_loss: 0.9544
Epoch [193/200], Step [631/1067], D_A_loss: 0.1275, D_B

Epoch [194/200], Step [261/1067], D_A_loss: 0.0246, D_B_loss: 0.0164, G_A_loss: 1.0696, G_B_loss: 0.9128


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
# Plot average losses
avg_losses = []
avg_losses.append(D_A_avg_losses)
avg_losses.append(D_B_avg_losses)
avg_losses.append(G_A_avg_losses)
avg_losses.append(G_B_avg_losses)
avg_losses.append(cycle_A_avg_losses)
avg_losses.append(cycle_B_avg_losses)
utils.plot_loss(avg_losses, params.num_epochs, save=True, save_dir=save_dir)

# Make gif
utils.make_gif(params.dataset, params.num_epochs, save_dir=save_dir)
# Save trained parameters of model
torch.save(G_A.state_dict(), model_dir + 'generator_A_param.pkl')
torch.save(G_B.state_dict(), model_dir + 'generator_B_param.pkl')
torch.save(D_A.state_dict(), model_dir + 'discriminator_A_param.pkl')
torch.save(D_B.state_dict(), model_dir + 'discriminator_B_param.pkl')